# Cachegrind 

Cachegrind simulates the first-level and last-level caches. The reason for this choice is that the last-level cache has the most influence on runtime, as it masks accesses to main memory. Furthermore, the L1 caches often have low associativity, so simulating them can detect cases where the code interacts badly with this cache (eg. traversing a matrix column-wise with the row length being a power of 2).

Therefore, Cachegrind always refers to the I1, D1 and LL (last-level) caches.

Cachegrind gathers the following statistics (abbreviations used for each statistic is given in parentheses):

- I cache reads (Ir, which equals the number of instructions executed), I1 cache read misses (I1mr) and LL cache instruction read misses (ILmr).

- D cache reads (Dr, which equals the number of memory reads), D1 cache read misses (D1mr), and LL cache data read misses (DLmr).

- D cache writes (Dw, which equals the number of memory writes), D1 cache write misses (D1mw), and LL cache data write misses (DLmw).

- Conditional branches executed (Bc) and conditional branches mispredicted (Bcm).

- Indirect branches executed (Bi) and indirect branches mispredicted (Bim).

Note that D1 total accesses is given by D1mr + D1mw, and that LL total accesses is given by ILmr + DLmr + DLmw.

On a modern machine, an L1 miss will typically cost around 10 cycles, an LL miss can cost as much as 200 cycles, and a mispredicted branch costs in the region of 10 to 30 cycles. 

LL miss rate is computed relative to the total number of memory accesses, not the number of L1 misses. I.e. it is (ILmr + DLmr + DLmw) / (Ir + Dr + Dw) not (ILmr + DLmr + DLmw) / (I1mr + D1mr + D1mw)

Ir : I cache reads (ie. instructions executed)
I1mr: I1 cache read misses
I2mr: L2 cache instruction read misses
Dr : D cache reads (ie. memory reads)
D1mr: D1 cache read misses
D2mr: L2 cache data read misses
Dw : D cache writes (ie. memory writes)
D1mw: D1 cache write misses
D2mw: L2 cache data write misses


In [1]:
import re
import numpy as np    
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
num_font = dict(size=20,color='#000000')
default_font = dict(family="Helvetica Neue",size=24,color='#000000')
#colors=px.colors.sequential.deep[2:]  # Candidate: Prism
colors=px.colors.sequential.Cividis[::-1][1:]
colorq=px.colors.qualitative.Dark24
color_def=['#838060', '#4f3222','#cab577','#7e4a35','#a79e84', '#563f46','#8ca3a3', '#484f4f', '#bd5734','#625750','#587e76', '#7a3b2e', '#a96e5b', '#98A762', '#443586']
marker_symbols=['circle','square','diamond','cross','triangle-up-open','pentagon','hexagram','star','hourglass', 'octagon', 'star-open-dot', 'hexagram-open', 'pentagon-open-dot', 'triangle-down-open','diamond-x-open']

In [2]:
vertice = [100,1000 ,5000, 10000 ,50000, 100000]
edge = [200,500,1000,2000,5000,10000,20000,25000,40000,50000,80000,100000,500000, 1000000]
func_alias = ['Baseline', 'Inline', 'InlineSIMD', 'V1', 'V1Mg', 'V1MgSIMD', 'V1MgSIMD2', 'V1MgRd', 'V1MgRdSIMD', 'V1MgBS', 'V1MgBSSIMD', 'V2Mg','V2MgSIMD', 'V2MgBS', 'V2MgBSSIMD', 'V2MgSearch_nofree']
len(func_alias)

16

In [3]:
# extract data from 'exact_cycle.txt'
datalist = []
header=['V', 'E', 'flops','flcomps', 'intops', 'intcomps', 'total', 'cycles','flops/cycle']
with open("./exact_cycle.txt", 'r') as f:
    for line in f:
        line = line.split()
        num = [int(x.replace(',', '').strip()) for x in line[:-3]]
        num.append(float(line[-3]))
        num.append(float(line[-2]))
        datalist.append(num)
res = pd.DataFrame(datalist, columns=header)
res = res.convert_dtypes()
res['E/V'] = res['E']/res['V']
res

,V,E,flops,flcomps,intops,intcomps,total,cycles,flops/cycle,E/V
0,100,200,4797,1973,3687,10608,21065,1.428210e+05,0.147492,2.0
1,100,200,4797,1973,3687,10608,21065,8.898400e+04,0.236728,2.0
2,100,200,4797,1973,3687,10608,21065,1.495780e+05,0.140829,2.0
3,100,200,4797,1973,3687,10608,21065,1.260840e+05,0.167071,2.0
4,100,200,4797,1973,3687,10608,21065,1.448340e+05,0.145443,2.0
...,...,...,...,...,...,...,...,...,...,...
459,100000,1000000,65709219,66030414,110269360,266252528,508261521,4.424210e+09,0.114882,10.0
460,100000,1000000,65709219,66030414,110269360,266252528,508261521,4.340860e+09,0.117088,10.0
461,100000,1000000,65709219,66030414,110269360,266252528,508261521,4.005200e+09,0.126900,10.0
462,100000,1000000,65709219,66030414,110269360,266252528,508261521,3.813540e+09,0.133278,10.0


In [4]:
# extract data from runtime.log
runtime = []
with open("./runtime.txt", 'r') as f:
    for line in f:
        line = line.split()
        runtime.extend(line[2:])
res['runtime'] = runtime

In [5]:
func_col = []
print(len(res)/len(func_alias))
for i in range(int(len(res)/len(func_alias))):
    func_col.extend(func_alias)
res['function'] = func_col
res

29.0


,V,E,flops,flcomps,intops,intcomps,total,cycles,flops/cycle,E/V,runtime,function
0,100,200,4797,1973,3687,10608,21065,1.428210e+05,0.147492,2.0,0.000117401,Baseline
1,100,200,4797,1973,3687,10608,21065,8.898400e+04,0.236728,2.0,5.53e-05,Inline
2,100,200,4797,1973,3687,10608,21065,1.495780e+05,0.140829,2.0,0.000108051,InlineSIMD
3,100,200,4797,1973,3687,10608,21065,1.260840e+05,0.167071,2.0,0.000112351,V1
4,100,200,4797,1973,3687,10608,21065,1.448340e+05,0.145443,2.0,7.67005e-05,V1Mg
...,...,...,...,...,...,...,...,...,...,...,...,...
459,100000,1000000,65709219,66030414,110269360,266252528,508261521,4.424210e+09,0.114882,10.0,1.84157,V2Mg
460,100000,1000000,65709219,66030414,110269360,266252528,508261521,4.340860e+09,0.117088,10.0,1.9244,V2MgSIMD
461,100000,1000000,65709219,66030414,110269360,266252528,508261521,4.005200e+09,0.126900,10.0,1.66807,V2MgBS
462,100000,1000000,65709219,66030414,110269360,266252528,508261521,3.813540e+09,0.133278,10.0,1.6318,V2MgBSSIMD


In [6]:
fig = go.Figure()
#/or idx, e in enumerate(ploted_edge):
same_ef = res.loc[(res['V'] == 10000)&(res['function']=='Baseline')]
fig.add_trace(go.Scatter(x=same_ef["E"], y=same_ef["flcomps"], name=f"flcomp",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[0],size=10),
                            line=dict(color=colors[0], width=3)))
fig.add_trace(go.Scatter(x=same_ef["E"], y=same_ef["flops"], name=f"flops",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[1],size=10),
                            line=dict(color=colors[3], width=3)))
fig.add_trace(go.Scatter(x=same_ef["E"], y=same_ef["intops"], name=f"intops",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[2],size=10),
                            line=dict(color=colors[6], width=3)))
fig.add_trace(go.Scatter(x=same_ef["E"], y=same_ef["intcomps"], name=f"intcomp",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[3],size=10),
                            line=dict(color=colors[8], width=3)))

fig.update_layout(title=dict(text="<b>Operation Counts vs. #E</b>", font=default_font),
                  xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2, nticks=5,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="nonnegative",nticks=5,
                             showline=False),
                  showlegend=False, # legend=dict(valign='top', traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=10, t=50),
                  plot_bgcolor="#eeeeee",
                  )
fig.show(renderer="plotly_mimetype")
fig.write_image("ops_v10000.pdf")

In [7]:
fig = go.Figure()

perf_func = ['V1Mg', 'V2Mg', 'V2MgBS', 'V1MgSIMD', 'V2MgSIMD','V2MgBSSIMD']
#perf_func = [ 'V1MgBS', 'V1MgBSSIMD' ]
colors = ['#e1cc55',   '#808080', '#669999', '#ad981f', '#000b1a','#006080']
colors.extend(color_def)

for j, func in enumerate(perf_func):
    same = res.loc[(res['function']==func)&(res['V']==10000)]
    fig.add_trace(go.Scatter(x=same['E/V'], y=(same["intops"]+same['flops'])/same['cycles'], name=f"{func}",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[j],size=8),
                            line=dict(color=colors[j], width=3)))
legend_font=dict(size=16,color='#000000')
fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",nticks=6,
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True, rangemode="tozero",nticks=8,
                             showline=False),
                  showlegend=False, 
                  legend=dict(font=legend_font, orientation='h', valign='top', x=0, y=1.3, yanchor='top',xanchor='left'),
                  margin=dict(autoexpand=False, l=60, r=40, t=10,b=60),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("simd_minor_v10000.pdf")

In [8]:
julia = []
julia_header = ['V', 'E', 'runtime']
with open("./julia_runtime.log", 'r') as f:
    for line in f:
        line = line.split()
        julia.append([int(line[0]), int(line[1]),float(line[2])])
julia_df = pd.DataFrame(julia, columns=julia_header)
julia_df['E/V'] = julia_df['E']/julia_df['V']

In [9]:
fig = go.Figure()

runtime_func = ['Inline', 'V1', 'V1MgSIMD',  'V1MgBSSIMD', 'V2MgBSSIMD']
# func_alias = ['Baseline', 'Inline', 'InlineSIMD', 'V1', 'V1Mg', 'V1MgSIMD', 'V1MgSIMD2', 'V1MgRd', 'V1MgRdSIMD', 'V1MgBS', 'V1MgBSSIMD', 'V2Mg','V2MgSIMD', 'V2MgBS', 'V2MgBSSIMD', 'V2MgSearch_nofree']
colors = ['#e1cc55', '#ad981f',  '#808080', '#669999',  '#006080', '#000b1a']
julia_same=julia_df.loc[julia_df['V']==5000]
fig.add_trace(go.Scatter(x=julia_same['E/V'], y=julia_same["runtime"],  name=f"Julia",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[13],size=10),
                            line=dict(color='#665c70', width=3)))
for j, func in enumerate(runtime_func):
    same = res.loc[(res['function']==func)&(res['V']==5000)]
    fig.add_trace(go.Scatter(x=same['E/V'], y=same["runtime"], name=f"{func}",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[j+1],size=8),
                            line=dict(color=colors[j+1], width=3)))
legend_font=dict(size=16,color='#000000')
fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",nticks=6,
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True, rangemode="tozero",nticks=8,
                             showline=False),
                  showlegend=True, legend=dict(font=legend_font, orientation='h', valign='top', x=0, y=1.3, 
                                               yanchor='top',xanchor='left'),
                  margin=dict(autoexpand=False, l=80, r=30, t=100,b=60),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("runtime_v5000.pdf")

In [11]:
fig = go.Figure()

#runtime_func = ['V1', 'V1MgSIMD',  'V1MgBSSIMD', 'V2MgBSSIMD']
runtime_func =  [ 'V1Mg', 'V1MgSIMD', 'V1MgRd', 'V1MgRdSIMD', 'V1MgBS', 'V1MgBSSIMD', 'V2Mg','V2MgSIMD', 'V2MgBS', 'V2MgBSSIMD']
colors = ['#e1cc55',  '#808080', '#669999',  '#006080', '#000b1a']
colors.extend(color_def)
julia_same=julia_df.loc[julia_df['V']==100000]
fig.add_trace(go.Scatter(x=julia_same['E/V'], y=julia_same["runtime"],  name=f"Julia",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[13],size=10),
                            line=dict(color='#665c70', width=3)))
for j, func in enumerate(runtime_func):
    same = res.loc[(res['function']==func)&(res['V']==100000)]
    fig.add_trace(go.Scatter(x=same['E/V'], y=same["runtime"], name=f"{func}",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[j+1],size=8),
                            line=dict(color=colors[j+1], width=3)))
legend_font=dict(size=16,color='#000000')
fig.update_layout(#title=dict(text="Performance [intops/cycle] vs. Average Degree", font=default_font),
                  xaxis=dict(showline=True, showgrid=False, rangemode="tozero",nticks=3,
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True, rangemode="nonnegative",nticks=6,
                             showline=False),
                  showlegend=True,
                  margin=dict(autoexpand=True, l=60, r=20, t=30,b=60),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()

In [12]:
res.loc[(res['V']==100000)&(res['E']==1000000)]

,V,E,flops,flcomps,intops,intcomps,total,cycles,flops/cycle,E/V,runtime,function
448,100000,1000000,65709219,66030414,110269360,266252528,508261521,7.999960e+11,0.000635,10.0,336.526,Baseline
449,100000,1000000,65709219,66030414,110269360,266252528,508261521,6.131240e+10,0.008290,10.0,25.7609,Inline
450,100000,1000000,65709219,66030414,110269360,266252528,508261521,7.446020e+10,0.006826,10.0,25.8785,InlineSIMD
451,100000,1000000,65709219,66030414,110269360,266252528,508261521,8.334630e+09,0.060982,10.0,2.49954,V1
452,100000,1000000,65709219,66030414,110269360,266252528,508261521,4.903130e+09,0.103661,10.0,2.0324,V1Mg
453,100000,1000000,65709219,66030414,110269360,266252528,508261521,4.751630e+09,0.106966,10.0,2.04253,V1MgSIMD
454,100000,1000000,65709219,66030414,110269360,266252528,508261521,1.888810e+09,0.269091,10.0,0.877544,V1MgSIMD2
455,100000,1000000,65709219,66030414,110269360,266252528,508261521,4.915550e+09,0.103399,10.0,2.05945,V1MgRd
456,100000,1000000,65709219,66030414,110269360,266252528,508261521,4.769760e+09,0.106559,10.0,2.13561,V1MgRdSIMD
457,100000,1000000,65709219,66030414,110269360,266252528,508261521,4.118860e+09,0.123398,10.0,1.77764,V1MgBS


In [13]:
import statistics

speedup_julia = []
speedup_baseline = []
func_alias = ['Baseline', 'Inline', 'Inline+simd', 'V1', 'V1Mg', 'V1MgSIMD', 'V1MgSIMD2', 'V1MgRd', 'V1MgRdSIMD', 'V1MgBS', 'V1MgBSSIMD', 'V2Mg','V2MgSIMD', 'V2MgBS', 'V2MgBSSIMD', 'V2MgSearch_nofree']

for i, row in res.iterrows():
    if row['function'] == 'V1MgRdSIMD':
        julia = julia_df.loc[(julia_df['V']==row['V'])&(julia_df['E']==row['E'])]
        sj = float(julia['runtime'])/float(row['runtime'])
        baseline = res.loc[(res['V']==row['V'])&(res['E']==row['E'])&(res['function']=='Baseline')]
        sb = float(baseline['runtime'])/float(row['runtime'])
        speedup_julia.append(sj)
        speedup_baseline.append(sb)

print(f"compared to julia: max: {max(speedup_julia)}, median: {statistics.median(speedup_julia)}, min: {min(speedup_julia)}")
print(f"compared to baseline: max: {max(speedup_baseline)}, median: {statistics.median(speedup_baseline)}, min: {min(speedup_baseline)}")

compared to julia: max: 2281.9982658574, median: 8.490342537638632, min: 1.2827387894058504
compared to baseline: max: 164.05770099654282, median: 11.7881855112931, min: 1.1186267877390401


In [14]:
import statistics

speedup_baseline_ops_cycle = []
func_alias = ['Baseline', 'Inline', 'Inline+simd', 'V1', 'V1Mg', 'V1MgSIMD', 'V1MgSIMD2', 'V1MgRd', 'V1MgRdSIMD', 'V1MgBS', 'V1MgBSSIMD', 'V2Mg','V2MgSIMD', 'V2MgBS', 'V2MgBSSIMD', 'V2MgSearch_nofree']

for i, row in res.iterrows():
    if row['function'] == 'V1MgRdSIMD':
        baseline = res.loc[(res['V']==row['V'])&(res['E']==row['E'])&(res['function']=='Baseline')]
        base_pf = float(baseline['intops']+baseline['flops'])/float(baseline['cycles'])
        curr = float(row['intops']+row['flops'])/float(row['cycles'])
        s = float(curr)/float(base_pf)
        print(f"baseline: {base_pf}, {row['function']}: {curr}, V: {row['V']}, E: {row['E']}, s:{s}")
        speedup_baseline_ops_cycle.append(s)

print(f"compared to baseline: max: {max(speedup_baseline_ops_cycle)}, median: {statistics.median(speedup_baseline_ops_cycle)}, min: {min(speedup_baseline_ops_cycle)}")

baseline: 0.05940302896632848, V1MgRdSIMD: 0.042088552647897806, V: 100, E: 200, s:0.7085253627682004
baseline: 0.06870596880607903, V1MgRdSIMD: 0.0732538862456993, V: 100, E: 500, s:1.0661939205377724
baseline: 0.06708671349426197, V1MgRdSIMD: 0.08549558409138189, V: 100, E: 1000, s:1.2744041202539227
baseline: 0.0721104628158346, V1MgRdSIMD: 0.10035749873258396, V: 100, E: 2000, s:1.3917189657885076
baseline: 0.012895429666868404, V1MgRdSIMD: 0.05070433763562585, V: 1000, E: 2000, s:3.931961861332781
baseline: 0.0215875000635366, V1MgRdSIMD: 0.05878243057141473, V: 1000, E: 5000, s:2.722984616023419
baseline: 0.01833588691840592, V1MgRdSIMD: 0.05539796612640634, V: 1000, E: 10000, s:3.0212864189731
baseline: 0.01961773008670336, V1MgRdSIMD: 0.0651572337781177, V: 1000, E: 20000, s:3.321344186618228
baseline: 0.021108317552295596, V1MgRdSIMD: 0.061631274524792334, V: 1000, E: 25000, s:2.919762523569281
baseline: 0.02128297705049524, V1MgRdSIMD: 0.08639818000317043, V: 1000, E: 40000, 

V2MgBS

compared to julia: max: 3695.9282721583936, median: 19.696940920256274, min: 2.2880402830595905

compared to baseline: max: 775.7476765681582, median: 19.141701850242146, min: 1.3720831069911974

V2MgBSSIMD

compared to julia: max: 5550.294898089245, median: 19.41691526143016, min: 2.1788746121853775

compared to baseline: max: 764.7190983187088, median: 18.068909434570468, min: 1.3431445727176219

In [16]:
datalist = []
header=['V','E','Ir','I1mr','ILmr','Dr','D1mr','DLmr','Dw','D1mw','DLmw','Bc','Bcm','Bi','Bim','function']
with open("./cache_deg5.log", 'r') as f:
    for line in f:
        line = line.split()
        num = [int(x.replace(',', '').strip()) for x in line[:-1]]
        num.append(line[-1])
        datalist.append(num)
res_mem = pd.DataFrame(datalist, columns=header)
res_mem['Data Read Miss'] = res_mem['D1mr']+res_mem['DLmr']
res_mem['Data Write Miss'] = res_mem['D1mw']+res_mem['DLmw']
res_mem['Instr Miss'] = res_mem['I1mr']+res_mem['ILmr']
res_mem['LL miss rate'] = (res_mem['ILmr'] + res_mem['DLmr'] + res_mem['DLmw']) / (res_mem['Ir'] + res_mem['Dr'] + res_mem['Dw']) *100
res_mem['mem'] = res_mem['Dr'] + res_mem['Dw']
res_mem['branch miss rate'] = res_mem['Bcm']/res_mem['Bc']*100
res_mem.convert_dtypes()
res_mem_cycle = [] 
res_mem_flops = []
res_mem_intops = []
res_mem_flcomps = []
res_mem_intcomps = []
res_mem_runtime = []
for idx, row in res_mem.iterrows():
    same = res.loc[(res['V']==row['V'])&(res['E']==row['E'])&(res['function']==row['function'])]
    res_mem_cycle.append(float(same['cycles']))
    res_mem_flops.append(float(same['flops']))
    res_mem_intops.append(float(same['intops']))
    res_mem_flcomps.append(float(same['flcomps']))
    res_mem_intcomps.append(float(same['intcomps']))
    res_mem_runtime.append(float(same['runtime']))
res_mem['flops'] = res_mem_flops
res_mem['cycles'] = res_mem_cycle
res_mem['flcomps'] = res_mem_flcomps
res_mem['intcomps'] = res_mem_intcomps
res_mem['intops'] = res_mem_intops
res_mem['runtime'] = res_mem_runtime
res_mem['flops/cycle'] = res_mem['flops']/res_mem['cycles']
res_mem['E/V'] = res_mem['E']/res_mem['V']

In [17]:
res_mem.to_csv("res_cache.csv")

In [18]:
avg_mem = []
avg_mem.extend(res_mem['mem'][0:13]/1000) #100, 500
avg_mem.extend(res_mem['mem'][13:26]/10) # 1000, 5000
avg_mem.extend(res_mem['mem'][26:39]/3) #5000, 25000
avg_mem.extend(res_mem['mem'][39:52]/1) #10000, 50000
avg_mem

[170421.0,
 82034.0,
 85395.0,
 92477.0,
 92449.109,
 91727.865,
 92754.0,
 77697.0,
 69309.0,
 94328.0,
 97517.0,
 89876.0,
 70933.0,
 11246710.0,
 1612436.0,
 1626351.0,
 1692827.0,
 1710763.8,
 1420316.0,
 1498246.0,
 1596826.3,
 1588880.0,
 1699437.6,
 1688310.0,
 1424548.0,
 1714243.0,
 258237083.0,
 11257017.0,
 11211569.0,
 11417867.0,
 11483316.333333334,
 9915066.0,
 10350604.0,
 10574339.333333334,
 10474330.0,
 11353009.333333334,
 11697398.0,
 9835469.0,
 11356633.0,
 1018227382.0,
 25487773.0,
 25275359.0,
 25558559.0,
 25610463.0,
 22298576.0,
 23348945.0,
 23331159.0,
 23275864.0,
 25331508.0,
 26314499.0,
 22036368.0,
 25190945.0]

In [19]:
res_mem['avg_mem'] = avg_mem

In [20]:
clip = res_mem.loc[(res_mem['V']==10000) & (res_mem['function'].isin(['Baseline','V1','V1Mg','V2Mg']))]
#V=10000, E=100000
cmp = clip[['D1mr', 'D1mw', 'Dr','Dw', 'LL miss rate', 'Bc', 'branch miss rate','runtime','function']]
cmp.to_csv('cmp.csv')

In [21]:
fig = go.Figure()
# for idx, e in enumerate(ploted_edge):
colors=px.colors.sequential.Cividis[::-1]

major_improvement = [ 'V1Mg', 'V1MgBS', 'V1MgBSSIMD']
colors = ['#e1cc55', '#ad981f',  '#808080', '#669999',  '#006080', '#000b1a']

for j, func in enumerate(major_improvement):
    same = res_mem.loc[res_mem['function']==func] # same['Data Read Miss']+same['Data Write Miss']
    fig.add_trace(go.Scatter(x=same["V"], y=(same['branch miss rate']), name=f"{func}",
                            mode='lines+markers', marker=dict(symbol=marker_symbols[j],size=10),
                            line=dict(color=colors[j], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True, rangemode="tozero",
                             showline=False),
                  showlegend=True, legend=dict(traceorder="normal"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
#fig.write_image("b-miss_deg5.pdf")

In [22]:
# flops/cycle vs oi
fig = go.Figure()

# func_alias = ['Inline', 'Inline+simd', 'Vec', 'VecMg', 'VecMgSIMD', 'VecMgSIMDcsum', 'VecMgRand', 'VecMgRandSIMD', 'VecStructMg','VecStructMgSIMD']

major_improvement = ['Baseline', 'Inline', 'V1', 'V1MgSIMD', 'V1MgBSSIMD', 'V2MgBSSIMD']
#colors=px.colors.sequential.Cividis[::-1]
colors = ['#e1cc55', '#ad981f',  '#808080', '#669999',  '#006080', '#000b1a']

for j, func in enumerate(major_improvement):
    same = res_mem.loc[(res_mem['function']==func)]
    fig.add_trace(go.Scatter(x=(same["intops"]+same["flops"])/same["avg_mem"],
                             y=(same["intops"]+same["flops"])/same['cycles'], 
                             name=f"{func}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[j+1],size=6),
                             line=dict(color=colors[j], width=1)))

txval=[0.001,0.002,0.003,0.004,0.005,0.01,0.02,0.03,0.04,0.05,0.1,0.2,0.3,0.4,0.5,1] 
txtxt=['' for i in range(len(txval))] 
txtxt[0]='0.001'
txtxt[5]='0.01'
txtxt[10]='0.1'
txtxt[-1]='1'

tyval=[0.0005,0.001,0.002,0.003,0.004,0.005,0.01, 0.02,0.03,0.04,0.05,0.1, 0.2,0.3,0.4,0.5,1] 
tytxt=['' for i in range(len(tyval))] 
tytxt[1]='0.001'
tytxt[6]='0.01'
tytxt[11]='0.1'
tytxt[-1]='1'
legend_font=dict(size=16,color='#000000')

fig.update_layout(xaxis=dict(showline=True, showgrid=True,
                             linecolor='rgb(0, 0, 0)', linewidth=2,rangemode="normal",
                             tickwidth=3, tickfont=num_font, ticks='outside',tickvals=txval,ticktext=txtxt),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,
                             rangemode="nonnegative", tickvals=tyval,ticktext=tytxt,
                             showline=True),
                  showlegend=True, legend=dict(orientation='h', valign='top', x=0, y=1.3, yanchor='top',xanchor='left',
                                     traceorder="normal", font=legend_font),
                  margin=dict(autoexpand=False, l=70, r=20, t=100,b=60),
                  plot_bgcolor="#eeeeee"
                  )
fig.update_xaxes(type="log")
fig.update_yaxes(type="log")


x_vals = [i*0.01 for i in range(50)]
y_vals = [4.16*i for i in x_vals]
fig.add_trace(go.Scatter(x=x_vals, y=y_vals, showlegend=False,
                            mode='lines',#marker=dict(symbol=marker_symbols[j],size=5),
                            line=dict(color="#8c8c8c", width=1))
                            )

fig.show()
#fig.write_image("intflops_roofline_deg5.pdf")